# Reading Data

ESA++ uses an **indexable interface** to read grid data with bracket
notation. Every query returns a pandas DataFrame with primary-key
columns included automatically.

```python
from esapp import GridWorkBench
from esapp.components import Bus, Gen, Load, Branch

wb = GridWorkBench("path/to/case.pwb")
```

In [1]:
from esapp import GridWorkBench
from esapp.components import Bus, Gen, Load, Branch, Shunt, Area, Zone
import numpy as np
import pandas as pd
import ast

with open('../../../examples/data/case.txt', 'r') as f:
    case_path = ast.literal_eval(f.read().strip())

wb = GridWorkBench(case_path)

'open' took: 9.8761 sec


The indexable interface supports four read patterns:

| Syntax | Returns |
|---|---|
| `wb[Bus]` | Key columns only (e.g. `BusNum`) |
| `wb[Bus, "BusPUVolt"]` | Keys + one field |
| `wb[Bus, ["BusPUVolt", "BusAngle"]]` | Keys + multiple fields |
| `wb[Bus, :]` | Keys + **every** defined field |

## Keys Only

Passing just the component type returns its primary-key columns.
Buses are identified by `BusNum` alone; generators have a compound
key (`BusNum`, `GenID`).

In [2]:
wb[Bus].head()

,BusNum
0,1
1,2
2,3
3,4
4,5


In [3]:
wb[Gen].head()

,BusNum,GenID
0,2,1
1,2,2
2,2,3
3,2,4
4,23,1


## Single Field

Add a field name to retrieve that column alongside the keys.

In [4]:
wb[Bus, "BusPUVolt"].head()

,BusNum,BusPUVolt
0,1,0.993545
1,2,0.991225
2,3,0.984548
3,4,0.978800
4,5,0.988985


## Multiple Fields

A list requests several columns at once.

In [5]:
wb[Gen, ["GenMW", "GenMVR", "GenStatus"]].head()

,BusNum,GenID,GenMVR,GenMW,GenStatus
0,2,1,0.80000,2.500000,Closed
1,2,2,0.80000,2.500000,Closed
2,2,3,0.80000,2.500000,Closed
3,2,4,0.80000,2.500000,Closed
4,23,1,0.04408,69.274741,Closed


## Enum-Based Fields

Instead of raw strings, use the component's enum attributes for
IDE autocomplete and typo protection.

In [6]:
wb[Bus, [Bus.BusName, Bus.BusPUVolt, Bus.BusAngle]].head()

,BusAngle,BusName,BusNum,BusPUVolt
0,-1.119907,ALOHA138,1,0.993545
1,-3.927372,ALOHA69,2,0.991225
2,-4.731145,FLOWER69,3,0.984548
3,-5.745870,WAVE69,4,0.978800
4,-2.069792,HONOLULU138,5,0.988985


## All Fields

The slice syntax `wb[Type, :]` retrieves every defined field.
Useful for exploration, but can produce wide DataFrames.

In [7]:
wb[Bus, :].shape

(37, 581)

## Working with Results

Every result is a standard pandas DataFrame, so normal filtering
and aggregation work directly.

In [8]:
# Filter generators by status
gens = wb[Gen, ["GenMW", "GenMVR", "GenStatus"]]
online = gens[gens["GenStatus"] == "Closed"]

In [9]:
# Aggregate load totals
loads = wb[Load, ["LoadMW", "LoadMVR"]]
loads[["LoadMW", "LoadMVR"]].sum()

LoadMW     1136.290004
LoadMVR       0.000000
dtype: float64